# 수정

- to_back_xpath가 전체 페이지에 다 사용할 수 없어서 try, exception을 통해 진행해보려 했으나 너무 많은 예외가 있어서 사용 불가
    - 그냥 drive.get()으로 이전 페이지를 불러오는 방식 사용

In [13]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# 크롬드라이버 서비스를 설정합니다.
service = Service('./chromedriver')
driver = webdriver.Chrome(service=service)

# 저장할 디렉터리 이름
output_dir = "NPP" # 국민의힘정당

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 데이터를 저장할 DataFrame 생성
df = pd.DataFrame(columns=['page_index', 'date', 'content'])

for page in range(145, 3542):
    # 페이지 접속
    driver.get(f'https://www.peoplepowerparty.kr/news/comment/BBSDD0001/?page={page}')
    for index in range(1, 11): # 각 페이지에 10개의 항목이 있다고 가정
        index_xpath = f"/html/body/div[2]/div[2]/div[2]/section/div[4]/table/tbody/tr[{index}]/td[3]/a"
        index_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, index_xpath)))
        index_element.click()
        
        date_xpath = f"/html/body/div[2]/div[2]/div[2]/section/div[2]/dl/dd[1]"
        # 내용 xpath
        content_xpath = f"/html/body/div[2]/div[2]/div[2]/section/div[2]/dl/dd[2]"
        
        # 작성일자와 내용 정보 가져오기
        date_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, date_xpath)))
        content_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, content_xpath)))
        date = date_element.text.strip()
        content = content_element.text.strip()
        # 정보를 DataFrame에 추가
        new_row = {'page_index': f'{page}_{index}', 'date': date, 'content': content}
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        to_back_xpath = "/html/body/div[2]/div[2]/div[2]/section/div[2]/div/a"
        back_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, to_back_xpath)))
        back_element.click()
        
    # 매 페이지마다 적당히 휴식
    time.sleep(0.5)

    # 종료 조건 검사
    if page == 3541 and index == 5:
        break

# DataFrame을 CSV 파일로 저장
df.to_csv(os.path.join(output_dir, 'scraped_data.csv'), index=False, encoding='utf-8')

driver.quit()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div[2]/div[2]/section/div[2]/dl/dd[2]/p[85]/span[8]/a"}
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00EA8893+48451]
	(No symbol) [0x00E3B8A1]
	(No symbol) [0x00D45058]
	(No symbol) [0x00D70467]
	(No symbol) [0x00D7069B]
	(No symbol) [0x00D9DD92]
	(No symbol) [0x00D8A304]
	(No symbol) [0x00D9C482]
	(No symbol) [0x00D8A0B6]
	(No symbol) [0x00D67E08]
	(No symbol) [0x00D68F2D]
	GetHandleVerifier [0x01108E3A+2540266]
	GetHandleVerifier [0x01148959+2801161]
	GetHandleVerifier [0x0114295C+2776588]
	GetHandleVerifier [0x00F32280+612144]
	(No symbol) [0x00E44F6C]
	(No symbol) [0x00E411D8]
	(No symbol) [0x00E412BB]
	(No symbol) [0x00E34857]
	BaseThreadInitThunk [0x760C00C9+25]
	RtlGetAppContainerNamedObjectPath [0x777E7B1E+286]
	RtlGetAppContainerNamedObjectPath [0x777E7AEE+238]


In [8]:
df

,page_index,date,content
0,1_1,작성일\n2023-07-18,오늘 열린 한미 양국간의 제1차 핵협의그룹(NCG) 회의는 대한민국의 안보체계에 새...
1,1_2,작성일\n2023-07-18,혹시나 했는데 역시나였다.\n\n가죽을 벗기고 뼈를 깎는 노력으로 당을 혁신하겠다는...
2,1_3,작성일\n2023-07-18,민주당이 오늘 의총에서 부랴부랴 혁신위 1호 쇄신안 ‘불체포특권 포기’를 추인했지만...
3,1_4,작성일\n2023-07-18,7월 18일 의원총회 주요내용은 다음과 같다.\n <윤재옥 원내대표>\n 일주일...
4,1_5,작성일\n2023-07-18,연일 계속된 폭우로 인해 인명사고가 속출했다. 어떤 말로도 위로가 되지 않겠지만 사...
...,...,...,...
1440,145_1,작성일\n2022-12-29,사업주뿐 아니라 근로자들도 간절히 원하고 있는 주 52시간을 8시간 초과해 연장 근...
1441,145_2,작성일\n2022-12-29,민주당이 노웅래 의원에 대한 체포동의안을 부결시키고 심지어 그 이유를 법무부 장관 ...
1442,145_3,작성일\n2022-12-29,"정진석 비상대책위원장, 해병대6여단 방문 주요내용 [보도자료]\n2022-12-27..."
1443,145_4,작성일\n2022-12-29,"정진석 비상대책위원장, 천안함 46용사 위령탑 참배 주요내용 [보도자료]\n2022..."


# 최종 국민의힘(NPP)

In [17]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# 크롬드라이버 서비스를 설정합니다.
service = Service('./chromedriver')
driver = webdriver.Chrome(service=service)

# 저장할 디렉터리 이름
output_dir = "NPP" # 국민의힘정당

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 데이터를 저장할 DataFrame 생성
df = pd.DataFrame(columns=['page_index', 'date', 'content'])

for page in range(1, 3542):
    # 페이지 접속
    driver.get(f'https://www.peoplepowerparty.kr/news/comment/BBSDD0001/?page={page}')
    for index in range(1, 11): # 각 페이지에 10개의 항목이 있다고 가정
        index_xpath = f"/html/body/div[2]/div[2]/div[2]/section/div[4]/table/tbody/tr[{index}]/td[3]/a"
        index_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, index_xpath)))
        index_element.click()
        
        date_xpath = f"/html/body/div[2]/div[2]/div[2]/section/div[2]/dl/dd[1]"
        # 내용 xpath
        content_xpath = f"/html/body/div[2]/div[2]/div[2]/section/div[2]/dl/dd[2]"
        
        # 작성일자와 내용 정보 가져오기
        date_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, date_xpath)))
        content_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, content_xpath)))
        date = date_element.text.strip()
        content = content_element.text.strip()
        # 정보를 DataFrame에 추가
        new_row = {'page_index': f'{page}_{index}', 'date': date, 'content': content}
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        driver.get(f'https://www.peoplepowerparty.kr/news/comment/BBSDD0001/?page={page}')

    # 매 페이지마다 적당히 휴식
    time.sleep(0.5)

    # 종료 조건 검사
    if page == 3541 and index == 5:
        break

# DataFrame을 CSV 파일로 저장
df.to_csv(os.path.join(output_dir, 'scraped_data.csv'), index=False, encoding='utf-8')

driver.quit()

In [35]:
df

,page_index,date,content
0,1_1,작성일\n2023-07-19,"김기현 당 대표는 2023. 7. 19.(수) 11:00, 신월 빗물저류배수시설에 ..."
1,1_2,작성일\n2023-07-19,"오늘 한 언론에 보도된 무소속 박완주 의원의 검찰 공소장 안에는, 강력 성범죄자의 ..."
2,1_3,작성일\n2023-07-19,7월 19일 원내대책회의 주요내용은 다음과 같다.\n <윤재옥 원내대표>\n 전...
3,1_4,작성일\n2023-07-19,더불어민주당이 윤석열 정부의 ‘시행령 추진’에 본격적으로 제동을 걸고 있다. \n\...
4,1_5,작성일\n2023-07-19,"그칠 줄 모르던 비가 잠시 소강상태에 접어들었지만, 폭우가 할퀴고 간 자리에 남겨진..."
...,...,...,...
35405,3541_6,작성일\n2004-03-25,전교조가 탄핵무효를 선언한 것은 탄핵수업 계획에 이어 교사의 본분을 망각한 행동으로...
35406,3541_7,작성일\n2004-03-25,지금 대한민국은 문자 그대로 토붕와해(土崩瓦解)의 위기상황이라 해도 과언이 아니다....
35407,3541_8,작성일\n2004-03-24,상임운영위원회의 주요내용은 다음과 같다\n\n<박근혜 당대표>\nㅇ 경제현황판을...
35408,3541_9,작성일\n2004-03-24,"전국공무원노조(전공노), 전국교직원노조(전교조)가 '탄핵반대' '탄핵무효' 선언을 ..."


In [36]:
import pandas as pd
from datetime import datetime

# 'date' 컬럼에서 '작성일\n' 제거
df['date'] = df['date'].str.replace('작성일\n', '', regex=False)

# 'date' 컬럼을 datetime 형태로 변환
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

# 'year'와 'quarter' 컬럼 생성
df['year'] = df['date'].dt.year
df['quarter'] = df['date'].dt.quarter

# 'content' 컬럼에서 '\n'와 '\t' 제거
df['content'] = df['content'].replace('\n', '', regex=True).replace('\t', '', regex=True)

In [37]:
df

,page_index,date,content,year,quarter
0,1_1,2023-07-19,"김기현 당 대표는 2023. 7. 19.(수) 11:00, 신월 빗물저류배수시설에 ...",2023,3
1,1_2,2023-07-19,"오늘 한 언론에 보도된 무소속 박완주 의원의 검찰 공소장 안에는, 강력 성범죄자의 ...",2023,3
2,1_3,2023-07-19,7월 19일 원내대책회의 주요내용은 다음과 같다. <윤재옥 원내대표> 전국적인 ...,2023,3
3,1_4,2023-07-19,더불어민주당이 윤석열 정부의 ‘시행령 추진’에 본격적으로 제동을 걸고 있다. KBS...,2023,3
4,1_5,2023-07-19,"그칠 줄 모르던 비가 잠시 소강상태에 접어들었지만, 폭우가 할퀴고 간 자리에 남겨진...",2023,3
...,...,...,...,...,...
35405,3541_6,2004-03-25,전교조가 탄핵무효를 선언한 것은 탄핵수업 계획에 이어 교사의 본분을 망각한 행동으로...,2004,1
35406,3541_7,2004-03-25,지금 대한민국은 문자 그대로 토붕와해(土崩瓦解)의 위기상황이라 해도 과언이 아니다....,2004,1
35407,3541_8,2004-03-24,상임운영위원회의 주요내용은 다음과 같다<박근혜 당대표>ㅇ 경제현황판을 만들어서 ...,2004,1
35408,3541_9,2004-03-24,"전국공무원노조(전공노), 전국교직원노조(전교조)가 '탄핵반대' '탄핵무효' 선언을 ...",2004,1


In [38]:
# 'date' 컬럼 삭제
df = df.drop('date', axis=1)

# 컬럼 순서 변경
df = df.reindex(columns=['page_index', 'year', 'quarter', 'content'])

In [39]:
df

,page_index,year,quarter,content
0,1_1,2023,3,"김기현 당 대표는 2023. 7. 19.(수) 11:00, 신월 빗물저류배수시설에 ..."
1,1_2,2023,3,"오늘 한 언론에 보도된 무소속 박완주 의원의 검찰 공소장 안에는, 강력 성범죄자의 ..."
2,1_3,2023,3,7월 19일 원내대책회의 주요내용은 다음과 같다. <윤재옥 원내대표> 전국적인 ...
3,1_4,2023,3,더불어민주당이 윤석열 정부의 ‘시행령 추진’에 본격적으로 제동을 걸고 있다. KBS...
4,1_5,2023,3,"그칠 줄 모르던 비가 잠시 소강상태에 접어들었지만, 폭우가 할퀴고 간 자리에 남겨진..."
...,...,...,...,...
35405,3541_6,2004,1,전교조가 탄핵무효를 선언한 것은 탄핵수업 계획에 이어 교사의 본분을 망각한 행동으로...
35406,3541_7,2004,1,지금 대한민국은 문자 그대로 토붕와해(土崩瓦解)의 위기상황이라 해도 과언이 아니다....
35407,3541_8,2004,1,상임운영위원회의 주요내용은 다음과 같다<박근혜 당대표>ㅇ 경제현황판을 만들어서 ...
35408,3541_9,2004,1,"전국공무원노조(전공노), 전국교직원노조(전교조)가 '탄핵반대' '탄핵무효' 선언을 ..."


In [40]:
df.to_csv(os.path.join(output_dir, 'final_NPP_df.csv'), index=False, encoding='utf-8')

In [41]:
df.iloc[1,3]

'오늘 한 언론에 보도된 무소속 박완주 의원의 검찰 공소장 안에는, 강력 성범죄자의 행태와 다름없는 추악한 모습이 고스란히 담겨있었다. 박 의원은 자신의 보좌관이었던 피해자를 노래방에서 강제로 추행하기 위해 비서를 먼저 내보내는 치밀함을 보였고,귀가하려는 피해자에게 차에 동승 할 것을 종용하고, 집에 도착해서는 피해자의 손목을 잡으며 자신의 집에서 술을 마시자는 등의 성범죄를 수차례 이어갔다.마치 사회면에 등장하는 악질 성범죄자의 모습을 보는 듯 하다. 게다가 사건이 발생했던 시기, 박 의원은 민주당의 정책위의장이자, 이재명 대선 후보 캠프의 정책본부장이었다. 이런 자가 여당의 정책위의장 자리에 앉아 있으니 제대로 된 정책이 나올 리 만무했고, 이런 자가 대선 캠프에서 직책을 맡았으니 민주당이 선거 내내 오만함과 국민무시로 일관했던 것이다. 이후에도 박 의원은 반성은커녕 "피해자가 금전적 보상을 원한다"는 2차 가해를 서슴지 않고, 수차례 면직시도를 하는 파렴치함까지 보였다. 박원순 전 시장의 성범죄 당시 “참혹하고 부끄럽다”던 박 의원이지만, 정작 자신이야말로 피해자에게 참혹한 범죄를 저지르고 일말의 부끄러움도 느끼지 않았던 것이다. 이제 박 의원은 민주당의 꼬리자르기 제명으로 무소속 신분이지만, 그가 민주당 소속으로 범죄를 저질렀고, 또 민주당의 국회의원과 정책위의장 지위를 이용한 권력형 성폭력의 가해자라는 사실은 변하지 않는다. 박 의원은 자신을 뽑아준 천안시민께 부끄럽지 않은가. 민주당은 이런 자에게 공천을 주어 국회에 발을 디디게 한 것이 국민께 죄송하지 않은가. 박 의원은 일말의 양심이라도 있다면 즉각 국회의원직을 사퇴하고 수사에 임하라. 민주당 역시 사과하고, 박 의원의 탈당 당시 약속했던 국회 차원의 징계에 나서길 바란다. 2023. 7. 19.국민의힘 수석부대변인 황 규 환'

In [ ]:
from datetime import datetime

# 끝부분의 날짜를 파싱하고 그 이후의 문자열을 삭제하는 함수 정의
def parse_and_remove_date(content):
    # 공백을 기준으로 문자열 분할
    split_content = content.split()

    for i in range(len(split_content)-1, -1, -1):
        # 날짜 문자열을 datetime 객체로 변환
        try:
            datetime.strptime(split_content[i], "%Y.%m.%d.")
            # 변환 가능한 경우 그 위치를 반환하고 루프 종료
            return ' '.join(split_content[:i])
        except ValueError:
            continue
    # 만약 날짜 형식의 문자열이 없는 경우 원래 문자열을 반환
    return content

# 각 컨텐츠에서 날짜 정보를 파싱하고 그 이후의 문자열을 삭제
df['content'] = df['content'].apply(parse_and_remove_date)

In [77]:
test_df = df.copy()

In [61]:
test_df.iloc[0,3]

'김기현 당 대표는 2023. 7. 19.(수) 11:00, 신월 빗물저류배수시설에 집중호우 대비 대심도 빗물터널 현장 방문을 했다. 주요내용은 다음과 같다.  <김기현 당 대표>  ‘빗물을 나르는 고속도로’라 불리는 대심도 빗물터널 조금 전에 둘러보고 왔다. 신월 빗물 전용 배수시설은 집중호우 때마다 주변 지역이 물바다가 되는 현상을 막고자 2013년 5월에 시작해서 2020년 5월까지 만 7년의 긴 기간 공사를 거쳐서 완공된 국내 최초의 대심도 빗물 저류 시설로 알고 있다.  물은 환경의 영역이기도 하지만 그보다 더 우선돼야 할 것은 치수의 영역이라고 저는 생각한다. 국가적으로 물길을 다스리는 모든 행정을 일컬어서 치수라고 하는데 이 치수가 제대로 되지 않으면 국민의 생명과 안전, 그리고 국민의 소중한 재산을 심대하게 침해하기 때문에 어떤 경우에도 목숨보다 더 우선되는 것이 없다는 차원에서 보면 치수가 가장 중요한 일이라고 말씀드리고 싶다. 특히 가뭄과 홍수에도 제대로 대응하려면 물은 치수의 차원에서 먼저 다뤄야 할 자원이라고 말씀드리고 싶다.  지난 정권 초기였던 2018년에 문재인 정부 시절입니다만, 물관리 일원화 조치를 한다고 하면서 국토해양부, 환경부, 농림축산식품부에 나눠졌던 물관리 업무를 환경부로 일원화하는 조치가 있었다. 그 당시에도 환경부가 전국의 지류·지천·하수 등의 물관리 업무 전반을 담당할 역량과 경험이 되는지에 대한 많은 의문이 제기되었지만, 이번 폭우 사태를 겪으면서 그 많은 의문이 현실로 드러나고 있는 것이란 생각을 하지 않을 수 없다.  물관리의 실패는 국민의 생명과 안전을 직접적 위협하고, 심대한 재산적 피해를 야기하는 것이니만큼 이 점에 대한 문제점의 보완이 절실하다고 하겠다. 과거 2010년과 2011년 당시에 서울 도심에 큰 물난리가 났다. 그때 대심도 빗물터널이 필요하다고 그런 필요성이 제기되었고 그 당시 시정을 책임지었던 서울시장이 오세훈 시장이었다. 그래서 오세훈 시장은 2021년까지 총 8,500억원 들여서 서울 도

# content 열의 맨 마지막 연도, 발언자 삭제 방법

In [44]:
from datetime import datetime

# 끝부분의 날짜를 파싱하고 그 이후의 문자열을 삭제하는 함수 정의
def parse_and_remove_date(content):
    # 공백을 기준으로 문자열 분할
    split_content = content.split()

    for i in range(len(split_content)-1, -1, -1):
        # 날짜 문자열을 datetime 객체로 변환
        try:
            datetime.strptime(split_content[i], "%Y.%m.%d.")
            # 변환 가능한 경우 그 위치를 반환하고 루프 종료
            return ' '.join(split_content[:i])
        except ValueError:
            continue
    # 만약 날짜 형식의 문자열이 없는 경우 원래 문자열을 반환
    return content

# 각 컨텐츠에서 날짜 정보를 파싱하고 그 이후의 문자열을 삭제
test_df['content'] = test_df['content'].apply(parse_and_remove_date)

In [49]:
def parse_and_remove_date(content):
    # 공백을 기준으로 문자열 분할
    split_content = content.split()

    for i in range(len(split_content)-1, -1, -1):
        # 날짜 문자열을 datetime 객체로 변환
        try:
            datetime.strptime(split_content[i], "%Y.%m.%d.")
            # 변환 가능한 경우 그 위치를 반환하고 루프 종료
            return ' '.join(split_content[:i])
        except ValueError:
            continue
    # 만약 날짜 형식의 문자열이 없는 경우 원래 문자열을 반환
    return content

# 각 컨텐츠에서 날짜 정보를 파싱하고 그 이후의 문자열을 삭제
test_df['content'] = test_df['content'].apply(parse_and_remove_date)

In [52]:
import datetime

def delete_date_from_end(content):
    split_content = content.split()
    for i in range(len(split_content)-1, -1, -1):
        for format in ["%Y. %m. %d.", "%Y.%m.%d."]:
            try:
                datetime.datetime.strptime(split_content[i], format)
                return ' '.join(split_content[:i])
            except ValueError:
                pass
    return content

test_df.iloc[:, 3] = test_df.iloc[:, 3].apply(delete_date_from_end)

KeyboardInterrupt: 

# 그냥 맨 마지막 20 이후 삭제하는 걸로

In [56]:
def delete_date_from_end(content):
    # '20'이 나오는 마지막 위치를 찾습니다
    last_index = content.rfind('20')

    # 만약 '20'이 없으면 원래의 내용을 반환합니다
    if last_index == -1:
        return content

    # 그렇지 않으면, '20'이 나오는 마지막 위치까지의 내용을 반환합니다
    return content[:last_index].strip()

test_df.iloc[:, 3] = test_df.iloc[:, 3].apply(delete_date_from_end)

In [68]:
test_df.iloc[303,3]

'‘교육개혁’은 그 무엇보다 대한민국의 미래가 달린 문제이다.공정과 정의의 원칙을 바로 세우기 위한 ‘교육개혁’은 더는 미룰 수 없는 시대적 과제이며 반드시 해야만 하는 필수 선택이다.그렇기에 ‘공교육에서 다루지 않는 문제의 출제는 교육당국이 사교육으로 내모는 것으로 불공정하고 부당함’을 언급한 윤석열 대통령의 발언은 학생들에 대한 ‘공교육 중심’‘기회의 공정과 균등’차원에서의 교육 개혁의 필요성을 강하게 언급한 것이다.‘개혁’은 늘 저항에 부딪혀 왔다. 그러나 우리 아이들의 미래를 위해 윤석열 정부는 역대 어느 정권보다 교육 개혁에 대한 확고한 의지를 가지고 있다.‘킬러 문항’은 학교에서 대비가 어렵기에 아이들을 사교육 시장으로 내몰고 있다.지난해 국내 사교육비는 26조 원에 이르고 있다. 사교육비를 감당할 수 있는 학부모의 경제적 능력이 자식들의 경쟁력이 되고, 학부모의 등골은 휘고 있다. 젊은이들은 자녀를 갖기 꺼리는 주요 이유로 사교육이 대두되고 있는 현실이다.사교육 이권 카르텔이라는 왜곡된 교육 현실을 바로 잡아 무너지고 있는 공교육을 정상화시키는 것이 교육 개혁의 본질이다.‘교육’만큼은 정쟁의 요소가 되어서는 안 된다.교육과정에 없는 킬러 문항 배제는 지난 대선 당시 이재명 대표의 공약이었고, 민주당도 사교육을 근절하고 공교육에 대한 신뢰를 높이기 위한 개혁의 시급함은 잘 알고 있을 것이다.연금·노동·교육 분야 개혁의 필요성에 국민 누구나 동의하지만, 추진함에 있어 각계의 다양한 목소리가 존재하는 것이 현실이다.그럼에도 지금까지 미루어 놓은 이 개혁들이 지금 추진되지 않는다면 우리 사회의 지속 가능성마저 위협받게 되는 중요한 시기이기에,국민의힘과 윤석열 정부는 우리가 직면한 위기와 도전을 기회로 삼고 충분한 설득과 사회적 합의를 이끌어 내는 등 개혁에 더욱 박차를 가해 나갈 것이다.2023. 6. 20.국민의힘 수석대변인 강 민 국'

In [62]:
# 원본 내용의 길이를 저장합니다
original_lengths = test_df.iloc[:, 3].apply(len)

# 수정된 내용의 길이를 얻습니다
new_lengths = test_df.iloc[:, 3].apply(delete_date_from_end).apply(len)

# 50자 이상 줄어든 행을 찾습니다
large_differences = (original_lengths - new_lengths) >= 50

# 그 위치를 출력합니다
print(test_df[large_differences])

      page_index  year  quarter  \
304         31_5  2023        2   
1057       106_8  2023        1   
1450       146_1  2022        4   
1459      146_10  2022        4   
1475       148_6  2022        4   
...          ...   ...      ...   
35187     3519_8  2004        2   
35269    3527_10  2004        2   
35271     3528_2  2004        2   
35343     3535_4  2004        2   
35345     3535_6  2004        2   

                                                 content  
304    결정적 변화존경하는 국민 여러분, 재외 동포 여러분!김진표 국회의장님과 선배 후배 ...  
1057   김기현 당 대표는 2023. 3. 20(월) 10:00, 국회 본관 국회의장실에서 ...  
1450   12월 26일 비상대책위원회의 주요내용은 다음과 같다. <정진석 비상대책위원장> 이...  
1459   12월 22일 비상대책위원회의 주요내용은 다음과 같다. <정진석 비상대책위원장> 문...  
1475   12월 23일 원내대책회의 주요내용은 다음과 같다. <주호영 원내대표> 법정 기한은...  
...                                                  ...  
35187  「개혁」과 「새정치」라는 말을 독점하다시피 해온 노무현 대통령과 열우당이 정작 자신...  
35269  이 프로는 9일밤 '색깔론 언제까지…'라는 주제를 다루면서 한나라당 대변인 본인 전...  
35271  청주시 우암동 소재 건물 내 윤락 업소 임대, 재산신고 누락 의혹, 부동산

In [67]:
df.iloc[1475,3]

'12월 23일 원내대책회의 주요내용은 다음과 같다. <주호영 원내대표> 법정 기한은 많이 넘겼지만, 연말까지 가지 않고 그래도 크리스마스 이전에 내년도 예산안이 통과될 수 있게 돼서 참으로 다행스럽게 생각한다. 2014년 국회법 개정 이후에는 12월 9일을 넘긴 적이 없었는데 이번에는 지키지 못하게 돼서 국민 여러분께 참으로 죄송스럽게 생각한다. 169석의 거대 민주당의 동의 없이는 예산안 하나 처리할 수가 없고 또 항목 하나하나마다 일일이 민주당의 동의를 받아야 했다. 처음에는 수십 가지 쟁점이 있었지만, 여러 차례 회동을 통해서 많이 줄어들었고, 마지막에 법인세 그다음에 경찰국과 인사정보관리단 예산, 그다음에 지역화폐 예산, 이 세 가지 문제 때문에 거의 10일 이상 끌어온 것 같다. 지역화폐는 윤석열 정부의 여러 가지 정책 방향이나 이런 것과 맞지 않아서 올해는 1원도 편성하지 않았지만, 이재명 민주당 대표가 7,050억원의 지역화폐 예산을 요구하는 바람에 그것 때문에 많이 오래 걸렸다. 어쩔 수 없이 민주당의 동의를 받아야 하기 때문에 절반인 3,525억 편성할 수밖에 없었다는 점 말씀을 드린다. 정부 당국은 일주일여밖에 남지 않은 짧은 기간이지만, 철저히 집행준비를 하고 낭비 요소가 없도록 어려운 시기에 재정이 경제의 마중물이 될 수 있도록 철저히 준비하고 낭비가 없도록 해주길 바라겠다. 저는 사법 절차에 정치권이 언급하는 자체가 적절하지 않다는 입장을 가지고 있지만, 성남FC 후원금과 관련해서 민주당 전체가 야당 탄압이라고 해서 제가 한번 들여다봤다. 성남FC 후원금은 성남에 연고를 둔 기업들이 후원금을 냈는데, 이례적으로 엄청나게 많은 돈들을 냈다. 두산건설이 45억, 농협이 50억, 네이버가 39억, 분당차병원이 33억, 현대백화점이 5억 6천, 알파돔시티가 5억 5천을 냈다. 합계 178억이다. 어느 FC 후원금을 기업들이 이렇게 몇십억씩 내는 경우가 있었는가. 이 기업들이 다른 곳에 후원을 이렇게 많이 하지 않았다. 그런데도 보니까 두산건설

In [72]:
def delete_date_from_end(content):
    # '20'이 나오는 마지막 위치를 찾습니다
    last_index = content.rfind('20')

    # 만약 '20'이 없으면 원래의 내용을 반환합니다
    if last_index == -1:
        return content

    # '20'이 나오는 마지막 위치까지의 내용을 얻습니다
    new_content = content[:last_index].strip()

    # 만약 원본 내용과 새 내용의 길이 차이가 20 이상이면 원래의 내용을 반환합니다
    if len(content) - len(new_content) >= 35:
        return content

    # 그렇지 않으면, 새 내용을 반환합니다
    return new_content

test_df.iloc[:, 3] = test_df.iloc[:, 3].apply(delete_date_from_end)

In [76]:
test_df.iloc[303,3]

'‘교육개혁’은 그 무엇보다 대한민국의 미래가 달린 문제이다.공정과 정의의 원칙을 바로 세우기 위한 ‘교육개혁’은 더는 미룰 수 없는 시대적 과제이며 반드시 해야만 하는 필수 선택이다.그렇기에 ‘공교육에서 다루지 않는 문제의 출제는 교육당국이 사교육으로 내모는 것으로 불공정하고 부당함’을 언급한 윤석열 대통령의 발언은 학생들에 대한 ‘공교육 중심’‘기회의 공정과 균등’차원에서의 교육 개혁의 필요성을 강하게 언급한 것이다.‘개혁’은 늘 저항에 부딪혀 왔다. 그러나 우리 아이들의 미래를 위해 윤석열 정부는 역대 어느 정권보다 교육 개혁에 대한 확고한 의지를 가지고 있다.‘킬러 문항’은 학교에서 대비가 어렵기에 아이들을 사교육 시장으로 내몰고 있다.지난해 국내 사교육비는 26조 원에 이르고 있다. 사교육비를 감당할 수 있는 학부모의 경제적 능력이 자식들의 경쟁력이 되고, 학부모의 등골은 휘고 있다. 젊은이들은 자녀를 갖기 꺼리는 주요 이유로 사교육이 대두되고 있는 현실이다.사교육 이권 카르텔이라는 왜곡된 교육 현실을 바로 잡아 무너지고 있는 공교육을 정상화시키는 것이 교육 개혁의 본질이다.‘교육’만큼은 정쟁의 요소가 되어서는 안 된다.교육과정에 없는 킬러 문항 배제는 지난 대선 당시 이재명 대표의 공약이었고, 민주당도 사교육을 근절하고 공교육에 대한 신뢰를 높이기 위한 개혁의 시급함은 잘 알고 있을 것이다.연금·노동·교육 분야 개혁의 필요성에 국민 누구나 동의하지만, 추진함에 있어 각계의 다양한 목소리가 존재하는 것이 현실이다.그럼에도 지금까지 미루어 놓은 이 개혁들이 지금 추진되지 않는다면 우리 사회의 지속 가능성마저 위협받게 되는 중요한 시기이기에,국민의힘과 윤석열 정부는 우리가 직면한 위기와 도전을 기회로 삼고 충분한 설득과 사회적 합의를 이끌어 내는 등 개혁에 더욱 박차를 가해 나갈 것이다.2023. 6.'

In [78]:
import re

def delete_date_from_end(content):
    # '20XX'를 찾는 정규 표현식을 정의합니다. 
    # rfind() 대신 re 라이브러리의 findall() 함수를 사용하면 모든 매치를 찾을 수 있습니다.
    matches = re.findall('20\d{2}', content)

    # 만약 '20XX'가 없으면 원래의 내용을 반환합니다
    if not matches:
        return content

    # '20XX'가 있는 마지막 위치를 찾습니다
    last_index = content.rfind(matches[-1])

    # '20XX'가 나오는 마지막 위치까지의 내용을 얻습니다
    new_content = content[:last_index].strip()

    # 만약 원본 내용과 새 내용의 길이 차이가 20 이상이면 원래의 내용을 반환합니다
    if len(content) - len(new_content) >= 35:
        return content

    # 그렇지 않으면, 새 내용을 반환합니다
    return new_content

test_df.iloc[:, 3] = test_df.iloc[:, 3].apply(delete_date_from_end)

In [85]:
test_df.iloc[2307, 3]

'7월 4일 최고위원회의 주요내용은 다음과 같다.  <권성동 원내대표>  여야 합의로 국회의장을 선출했던 것은 1987년부터 이어온 우리 국회의 오랜 전통이다. 2008년 18대 국회 당시 야당인 통합민주당은 81석에 불과했지만, 김형오 국회의장 내정자는 단독으로 국회를 소집하지 않고 여야 합의를 기다렸다. 그런데 민주당은 21대 국회 전반기에 이어서 또다시 의장단을 단독으로 선출하겠다고 공헌하였다. 절대다수 의석을 가진 민주당의 오만과 독선의 결과이다.  국회법상 본회의 단독 소집 근거는 없다. 국회의장 공석 시 본회의 개의와 안건은 오직 교섭단체 간 합의로만 정할 수 있다. 민주당의 단독 본회의 소집, 단독 선출을 명백한 불법이다. 민주당이 일방적으로 데드라인을 정해놓고 소수당을 압박하는 것, 역시 온당치 못한 태도지만 우리 국민의힘은 어떻게든 국회를 정상화해야 한다는 절박한 심정으로 최대한 양보하며 협상에 임했다.  하지만 민주당은 원구성 협상과 전혀 무관한 검수완박 합의 이행을 전제조건으로 내세웠다. 우리당으로서는 결코 수용할 수 없는 내용이다. 사개특위 구성은 검수완박에 따른 후속조치이다. 검수완박 합의는 이미 파기되었다. 파기된 합의를 이행할 이유가 없거니와 사개특위 구성에 동의하는 것은 민주당의 검수완박 강행처리에 면죄부를 주는 것밖에 되지 않는다.  우리당은 물론 정부도 검수완박 강행처리 과정의 불법성, 위헌성에 대하여 헌법재판소의 권한쟁의 및 효력정지가처분을 제기하였고 헌재의 판단을 기다리고 있다. 설령 구성을 논의하더라도 헌재의 결정 뒤로 미루는 것이 합리적이다. 이에 협상 과정에서 사개특위 문제는 헌법재판소의 결정 뒤에 논의하자 그것마저도 어렵다면은 사개특위 구성은 여야 5:5로 하고 위원장은 국민의힘이 맡도록 하자는 절충안을 제시했지만, 민주당은 끝내 거부하였다. 민생을 볼모로 삼아 우리 국민의힘을 검수완박 공보물로 만들고 사개특위마저 힘의 논리로 일방 운영하겠다는 것 아닌 이상 거부하는 이유와 명분을 찾기 어렵다. 민주당은 앞에서는 민생

In [84]:
df.iloc[2307,3]

'7월 4일 최고위원회의 주요내용은 다음과 같다.  <권성동 원내대표>  여야 합의로 국회의장을 선출했던 것은 1987년부터 이어온 우리 국회의 오랜 전통이다. 2008년 18대 국회 당시 야당인 통합민주당은 81석에 불과했지만, 김형오 국회의장 내정자는 단독으로 국회를 소집하지 않고 여야 합의를 기다렸다. 그런데 민주당은 21대 국회 전반기에 이어서 또다시 의장단을 단독으로 선출하겠다고 공헌하였다. 절대다수 의석을 가진 민주당의 오만과 독선의 결과이다.  국회법상 본회의 단독 소집 근거는 없다. 국회의장 공석 시 본회의 개의와 안건은 오직 교섭단체 간 합의로만 정할 수 있다. 민주당의 단독 본회의 소집, 단독 선출을 명백한 불법이다. 민주당이 일방적으로 데드라인을 정해놓고 소수당을 압박하는 것, 역시 온당치 못한 태도지만 우리 국민의힘은 어떻게든 국회를 정상화해야 한다는 절박한 심정으로 최대한 양보하며 협상에 임했다.  하지만 민주당은 원구성 협상과 전혀 무관한 검수완박 합의 이행을 전제조건으로 내세웠다. 우리당으로서는 결코 수용할 수 없는 내용이다. 사개특위 구성은 검수완박에 따른 후속조치이다. 검수완박 합의는 이미 파기되었다. 파기된 합의를 이행할 이유가 없거니와 사개특위 구성에 동의하는 것은 민주당의 검수완박 강행처리에 면죄부를 주는 것밖에 되지 않는다.  우리당은 물론 정부도 검수완박 강행처리 과정의 불법성, 위헌성에 대하여 헌법재판소의 권한쟁의 및 효력정지가처분을 제기하였고 헌재의 판단을 기다리고 있다. 설령 구성을 논의하더라도 헌재의 결정 뒤로 미루는 것이 합리적이다. 이에 협상 과정에서 사개특위 문제는 헌법재판소의 결정 뒤에 논의하자 그것마저도 어렵다면은 사개특위 구성은 여야 5:5로 하고 위원장은 국민의힘이 맡도록 하자는 절충안을 제시했지만, 민주당은 끝내 거부하였다. 민생을 볼모로 삼아 우리 국민의힘을 검수완박 공보물로 만들고 사개특위마저 힘의 논리로 일방 운영하겠다는 것 아닌 이상 거부하는 이유와 명분을 찾기 어렵다. 민주당은 앞에서는 민생

In [86]:
test_df.to_csv(os.path.join(output_dir, 'final_NPP.csv'), index=False, encoding='utf-8')